In [ ]:
# pip install pandas requests
import io, requests
import pandas as pd
import numpy as np

# ---------------------------
# Helper to load XPT files
# ---------------------------
def get_xpt(url: str) -> pd.DataFrame:
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=60)
    r.raise_for_status()
    b = r.content
    #XPORT header signature
    if b"HEADER RECORD*******LIBRARY HEADER RECORD" not in b[:200]:
        raise ValueError("Got non-XPORT content from:\n" + url)
    return pd.read_sas(io.BytesIO(b), format="xport")

# ---------------------------
# NHANES 2017–2018 file locations
# ---------------------------
root = "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/"
urls = {
    "DEMO":   root + "DEMO_J.xpt",     # Demographics
    "DR1TOT": root + "DR1TOT_J.xpt",   # Dietary totals, Day 1
    "DR2TOT": root + "DR2TOT_J.xpt",   # Dietary totals, Day 2
    "PAQ":    root + "PAQ_J.xpt",      # Physical activity
    "SMQ":    root + "SMQ_J.xpt",      # Smoking
    "ALQ":    root + "ALQ_J.xpt",      # Alcohol
    "SLQ":    root + "SLQ_J.xpt",      # Sleep
    "BMX":    root + "BMX_J.xpt",      # Body measures (height, weight)
    "RXQ":    root + "RXQ_RX_J.xpt",   # Medications (RXDCOUNT)
}

dfs = {k: get_xpt(u) for k, u in urls.items()}

# ---------------------------
# 1. Demographics
# ---------------------------
demo = dfs["DEMO"][[
    "SEQN",
    "RIAGENDR",   # sex
    "RIDAGEYR",   # age
    "RIDRETH3",   # race/ethnicity
    "INDFMPIR",   # poverty ratio
    "DMDEDUC2"    # education level (adults)
]].rename(columns={
    "RIAGENDR": "sex_code",
    "RIDAGEYR": "age_years",
    "RIDRETH3": "race_eth",
    "INDFMPIR": "poverty_ratio",
    "DMDEDUC2": "education_level_raw"
})

# sex label
demo["sex"] = demo["sex_code"].map({1: "Male", 2: "Female"})

# clean education: set 7, 9 (Refused/Don’t know) to NaN
demo["education_level"] = demo["education_level_raw"].replace({7: np.nan, 9: np.nan})

# ---------------------------
# 2. Diet (Day 1 + Day 2)
# ---------------------------
nutr = ["KCAL", "PROT", "TFAT", "CARB", "SUGR", "FIBE", "CHOL"]

dr1 = dfs["DR1TOT"][["SEQN", "WTDRD1"] +
                    [f"DR1T{n}" for n in nutr if f"DR1T{n}" in dfs["DR1TOT"].columns]]
dr2 = dfs["DR2TOT"][["SEQN", "WTDR2D"] +
                    [f"DR2T{n}" for n in nutr if f"DR2T{n}" in dfs["DR2TOT"].columns]]

diet = dr1.merge(dr2, on="SEQN", how="left")

# average Day1 & Day2 when both present, else Day1
for n in nutr:
    c1, c2 = f"DR1T{n}", f"DR2T{n}"
    if c1 in diet.columns:
        if c2 in diet.columns:
            diet[f"{n}_AVG"] = diet[[c1, c2]].mean(axis=1, skipna=True)
        else:
            diet[f"{n}_AVG"] = diet[c1]

# keep original D1/D2 names
diet = diet.rename(columns={
    **{f"DR1T{n}": f"{n}_D1" for n in nutr if f"DR1T{n}" in diet.columns},
    **{f"DR2T{n}": f"{n}_D2" for n in nutr if f"DR2T{n}" in diet.columns},
})

# ---------------------------
# 3. Lifestyle and health
# ---------------------------

# Physical activity: rename to representative names
paq = dfs["PAQ"][["SEQN", "PAQ605", "PAQ620", "PAQ635", "PAD680"]].rename(columns={
    "PAQ605": "vigorous_work_activity",
    "PAQ620": "moderate_work_activity",
    "PAQ635": "walk_bike_minutes_week",
    "PAD680": "sedentary_minutes_day"
})

# Smoking
smq = dfs["SMQ"][["SEQN", "SMQ020", "SMQ040", "SMD030"]].rename(columns={
    "SMQ020": "ever_smoked_100_cigs",
    "SMQ040": "current_smoking_freq",
    "SMD030": "days_smoked_past_30"
})

# Alcohol
alq = dfs["ALQ"][["SEQN", "ALQ121", "ALQ130", "ALQ151", "ALQ142"]].rename(columns={
    "ALQ121": "alcohol_days_year",
    "ALQ130": "drinks_per_drinking_day",
    "ALQ151": "binge_drinking_freq",
    "ALQ142": "heavy_drinking_freq"
})

# Sleep
sleep_vars = ["SEQN", "SLD012", "SLQ030", "SLQ040", "SLQ050", "SLQ120"]
slq = dfs["SLQ"][[c for c in sleep_vars if c in dfs["SLQ"].columns]].rename(columns={
    "SLD012": "sleep_hours",
    "SLQ030": "trouble_sleeping",
    "SLQ040": "told_sleep_disorder",
    "SLQ050": "dx_sleep_disorder",
    "SLQ120": "sleep_med_use"
})

# Body measures – rename weight and height
bmx = dfs["BMX"][["SEQN", "BMXWT", "BMXHT"]].rename(columns={
    "BMXWT": "weight_kg",
    "BMXHT": "height_cm"
})

# Medications – number of prescription meds
rxq = dfs["RXQ"][["SEQN", "RXDCOUNT"]].rename(columns={
    "RXDCOUNT": "n_prescription_meds"
})

# ---------------------------
# 4. Merge everything
# ---------------------------
out = demo.merge(diet, on="SEQN", how="inner")
out = (
    out.merge(paq, on="SEQN", how="left")
       .merge(smq, on="SEQN", how="left")
       .merge(alq, on="SEQN", how="left")
       .merge(slq, on="SEQN", how="left")
       .merge(bmx, on="SEQN", how="left")
       .merge(rxq, on="SEQN", how="left")
)

# ---------------------------
# 5. Select ML columns
# ---------------------------
ml_cols = [
    "SEQN",
    "age_years",
    "sex",
    "sex_code",
    "race_eth",
    "poverty_ratio",
    "education_level",
    "WTDRD1",                  # diet sample weight
    # key diet var for modeling
    "KCAL_AVG",                # average daily calories
    # lifestyle & health
    "sleep_hours",
    "walk_bike_minutes_week",
    "sedentary_minutes_day",
    "days_smoked_past_30",
    "alcohol_days_year",
    "n_prescription_meds",
    # outcome & body size
    "height_cm",
    "weight_kg"
] + [c for c in out.columns if c.endswith("_AVG")]  # keep all nutrient averages too

ml = out[ml_cols].copy()

# ---------------------------
# 6. Basic cleaning filters
# ---------------------------
# drop any rows with missing values in selected columns
ml = ml.dropna()

# keep age >= 1 year
ml = ml[ml["age_years"] >= 1]

# keep valid poverty ratio
ml = ml[ml["poverty_ratio"] >= 0.01]

# save core and cleaned ML dataset
ml.to_csv("nhanes_2017_2018_ml_dataset.csv", index=False)
print("Saved nhanes_2017_2018_ml_dataset.csv with shape:", ml.shape)
print(ml.head())



Saved nhanes_2017_2018_ml_dataset.csv with shape: (4895, 24)
       SEQN  age_years     sex  sex_code  race_eth  poverty_ratio  \
2   93705.0       66.0  Female       2.0       4.0           0.82   
3   93705.0       66.0  Female       2.0       4.0           0.82   
4   93705.0       66.0  Female       2.0       4.0           0.82   
17  93713.0       67.0    Male       1.0       3.0           2.65   
18  93714.0       54.0  Female       2.0       4.0           1.86   

    education_level         WTDRD1  KCAL_AVG  sleep_hours  ...  \
2               2.0    7185.560737    1218.5          8.0  ...   
3               2.0    7185.560737    1218.5          8.0  ...   
4               2.0    7185.560737    1218.5          8.0  ...   
17              3.0  252846.565438    2103.5          5.5  ...   
18              4.0    9694.007642    1833.0          7.0  ...   

    n_prescription_meds  height_cm  weight_kg  KCAL_AVG  PROT_AVG  TFAT_AVG  \
2                   3.0      158.3       79.5   

In [ ]:
# ===========================
# 0. Imports
# ===========================
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# ===========================
# 1. Load cleaned ML dataset
# ===========================

ml = pd.read_csv("nhanes_2017_2018_ml_dataset.csv")

# ===========================
# 2. Define features and target
# ===========================

feature_cols = [
    "age_years",
    "sex_code",                 # numeric sex code
    "education_level",
    "poverty_ratio",
    "KCAL_AVG",                 # average daily calories
    "alcohol_days_year",
    "sleep_hours",
    "walk_bike_minutes_week",
    "days_smoked_past_30",
    "n_prescription_meds",
    "height_cm"
]

target_col = "weight_kg"

# Keep only the rows with complete data for all predictors + target
df = ml.copy()
X = df[feature_cols].copy()
y = df[target_col].copy()

mask = X.notna().all(axis=1) & y.notna()
X = X[mask]
y = y[mask]

print("Predicting weight_kg")
print("Features:", feature_cols)
print("X shape:", X.shape)

# display names for printing later
pretty_names = {
    "age_years": "Age (years)",
    "sex_code": "Sex (1=Male, 2=Female)",
    "education_level": "Education Level",
    "poverty_ratio": "Family Poverty-Income Ratio",
    "KCAL_AVG": "Average Daily Calorie Intake (kcal)",
    "alcohol_days_year": "Alcohol Drinking Days per Year",
    "sleep_hours": "Average Sleep Duration (hours)",
    "walk_bike_minutes_week": "Walking/Biking Minutes per Week",
    "days_smoked_past_30": "Days Smoked in Past 30 Days",
    "n_prescription_meds": "Number of Prescription Medications",
    "height_cm": "Height (cm)"
}

# ===========================
# 3. Train/test split
# ===========================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=101
)

# ===========================
# 4. Linear Regression
# ===========================
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_lr = lr.predict(X_test)

print("\nLinear Regression Performance")
print("-----------------------------")
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_lr)))
print("R²:", r2_score(y_test, y_pred_lr))

lr_coef = pd.DataFrame({
    "feature": feature_cols,
    "coefficient": lr.coef_
})

# Add labels for readability
lr_coef["feature_full_title"] = lr_coef["feature"].map(pretty_names)
lr_coef = lr_coef.sort_values(by="coefficient", ascending=False)

print("\nLinear Regression Coefficients:")
print(lr_coef)

# ===========================
# 5. Random Forest Regression
# ===========================
rf = RandomForestRegressor(
    n_estimators=500,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("\nRandom Forest Performance")
print("-------------------------")
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_rf)))
print("R²:", r2_score(y_test, y_pred_rf))

rf_imp = pd.DataFrame({
    "feature": feature_cols,
    "importance": rf.feature_importances_
})
rf_imp["feature_full_title"] = rf_imp["feature"].map(pretty_names)
rf_imp = rf_imp.sort_values(by="importance", ascending=False)

print("\nRandom Forest Feature Importances:")
print(rf_imp)


Predicting weight_kg
Features: ['age_years', 'sex_code', 'education_level', 'poverty_ratio', 'KCAL_AVG', 'alcohol_days_year', 'sleep_hours', 'walk_bike_minutes_week', 'days_smoked_past_30', 'n_prescription_meds', 'height_cm']
X shape: (4895, 11)

Linear Regression Performance
-----------------------------
RMSE: 21.849361317989022
R²: 0.20143242307452014

Linear Regression Coefficients:
                   feature  coefficient                   feature_full_title
1                 sex_code     4.222179               Sex (1=Male, 2=Female)
7   walk_bike_minutes_week     3.638914      Walking/Biking Minutes per Week
10               height_cm     1.123923                          Height (cm)
9      n_prescription_meds     1.043510   Number of Prescription Medications
5        alcohol_days_year     0.428291       Alcohol Drinking Days per Year
4                 KCAL_AVG     0.000449  Average Daily Calorie Intake (kcal)
8      days_smoked_past_30    -0.006615          Days Smoked in Past 30 